In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import LightningModule, Trainer
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from torch import optim
from tqdm import tqdm
import torch.nn.functional as F
import numpy as np
import torch
import math
import timm

import transforms.eft as Eft
import datasets.radioml_dataset as RadiomlDT
from datasets.transformed_dataset import TransformedDataset
from utils import plot_confusion_matrix
from models.radio_rec_model import RadioRecNetwork
from models.resnet import create_resnet_2d

In [ ]:
print(torch.cuda.is_available())

In [ ]:
import datasets.signal_file_dataset as sfds

root = "/home/zwlin/data/radio"
sig_file_set = sfds.SignalFileSet(root, 'bin')
sig_file_set.print_info()

In [ ]:
sig_file_conf = sfds.SignalFileConfig()
sig_file_conf.fs = 100000 # 100K Hz sample rate
sig_file_conf.seg_len = 500 # 500 ms
sig_file_conf.seg_shift = 500 # 500 ms, no overlap
sig_file_conf.max_num = 1000
sig_file_conf.is_complex = True
sig_file_conf.energy_threshold = 6

sig_dataset = sfds.SignalDataSet(sig_file_set, sig_file_conf)
sig_dataset.print_info()

In [ ]:
class_list = sig_dataset.class_list
num_classes = len(class_list)

In [ ]:
# Repeat but plot andom sampling of the data
from datasets.visualize import IQVisualizer

example_dataloader = DataLoader(
    dataset=sig_dataset,
    batch_size=16,
    num_workers=10,
    shuffle=True,
    drop_last=True,
)
visualizer = IQVisualizer(
    data_loader = example_dataloader,
    visualize_target_transform = lambda target: [label_list[int(index)] for index in target]
)

for figure in iter(visualizer):
    figure.set_size_inches(14, 9)
    plt.show()
    break

In [ ]:
from transforms.transforms import Compose
from transforms.spectrogram_transform import SpectrogramConfig, Spectrogram

interleave_to_complex = Eft.InterleaveToComplex()

spectrogram_config = SpectrogramConfig()
spectrogram_config.nperseg = 256
spectrogram_config.noverlap = 128
spectrogram_config.nfft = 256
spectrogram_config.window = 'hann'
spectrogram_config.return_onesided = False
spectrogram_config.mode = 'magnitude'

spectorgram_transform = Spectrogram(spectrogram_config)

ds = TransformedDataset(sig_dataset, transform = Compose([interleave_to_complex, spectorgram_transform]))

In [ ]:
num_total = len(ds)
num_train = math.floor(0.8 * num_total)
num_val = math.floor(0.1 * num_total)
num_test = num_total - num_train - num_val

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(ds, [num_train, num_val, num_test],
                                                                         generator = torch.Generator().manual_seed(42))
idx = np.random.randint(len(train_dataset))
data, label = train_dataset[idx]
channels = data.shape[0]
print("Dataset length: {}".format(len(ds)))
print("Data shape: {}".format(data.shape))
print(f"Channels: {channels}")
print("Label Index: {}".format(label))
print("Label Class: {}".format(label_list[label]))

In [ ]:
# ----
# ### Format Dataset for Training
# Next, the datasets are then wrapped as `DataLoaders` to prepare for training.

# Create dataloaders
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=16,
    num_workers=8,
    shuffle=True,
    drop_last=True,
)
val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=16,
    num_workers=8,
    shuffle=False,
    drop_last=False,
)
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=16,
    num_workers=8,
    shuffle=False,
    drop_last=False,
)

In [ ]:
model = create_resnet_2d(name = 'resnet18', pretrained = False, num_classes = num_classes, in_chans = channels)
model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Training Config
learning_rate=0.002
epochs = 50

radiorec_model = RadioRecNetwork(model, learning_rate = learning_rate)

In [ ]:
# ----
# ### Train the Model
# To train the model, we first create a `ModelCheckpoint` to monitor the validation loss over time and save the best model as we go. The network is then instantiated and passed into a `Trainer` to kick off training.
import os
checkpoint_dir=f'exp/radioml2018-iq-all-{rediorec_model.learning_rate}-v3/checkpoints'
if not os.path.exists(checkpoint_dir):
    print(f'mkdir: {checkpoint_dir}')
    os.makedirs(checkpoint_dir)

In [ ]:
# Setup checkpoint callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath=checkpoint_dir,
    filename='{epoch}-{val_loss:.2f}',
    save_top_k=3,
    save_last=True,
    verbose=True,
    monitor="val_loss",
    mode="min",
)

In [ ]:
# Create and fit trainer

trainer = Trainer(
    max_epochs=epochs, callbacks=checkpoint_callback, devices=1, accelerator="gpu"
)

trainer.fit(radiorec_model, train_dataloaders=train_dataloader, val_dataloaders = val_dataloader)

In [ ]:
checkpoint_callback.best_model_path

In [ ]:
# After the model is trained, the checkpoint's weights are loaded into the model and the model is put into evaluation mode. The validation set is looped through, inferring results for each example and saving the predictions and the labels. Finally, the labels and predictions are passed into our confusion matrix plotting function to view the results and also passed into the `sklearn.metrics.classification_report` method to print metrics of interest.
# checkpoint_filename = checkpoint_callback.best_model_path
checkpoint_filename = 'exp/radioml2018-iq-all-0.002-v3/checkpoints/epoch=26-val_loss=0.13.ckpt'

# Load best checkpoint
device = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint = torch.load(
    checkpoint_filename, map_location=lambda storage, loc: storage
)

model = create_resnet(name = 'resnet18', pretrained = False, num_classes = num_classes, in_chans = channels)
test_model = RadioRecNetwork(model, learning_rate = learning_rate)
test_model.load_state_dict(checkpoint["state_dict"])
test_model = test_model.to(device=device).eval()

In [ ]:
# ### Evaluate the val set

# Infer results over validation set
num_val_examples = len(val_dataset)
y_raw_preds = np.empty((num_val_examples, num_classes))
y_preds = np.zeros((num_val_examples,))
y_true = np.zeros((num_val_examples,))

for i in tqdm(range(0, num_val_examples)):
    # Retrieve data
    idx = i  # Use index if evaluating over full dataset
    data, label = val_dataset[idx]
    # Infer
    data = torch.from_numpy(np.expand_dims(data, 0)).float().to(device)
    pred_tmp = test_model.predict(data)
    pred_tmp = pred_tmp.cpu().numpy() if torch.cuda.is_available() else pred_tmp
    # Argmax
    y_preds[i] = np.argmax(pred_tmp)
    # Store label
    y_true[i] = label

In [ ]:
acc = np.sum(np.asarray(y_preds) == np.asarray(y_true)) / len(y_true)
plot_confusion_matrix(
    y_true,
    y_preds,
    classes = label_list,
    normalize = True,
    title="Validation Set Modulations Confusion Matrix\nTotal Accuracy: {:.2f}%".format(
        acc * 100
    ),
    text=False,
    rotate_x_text=90,
    figsize=(16, 9),
)
plt.savefig(f"{checkpoint_dir}/val_result.png")

print("Val set classification report:")
print(classification_report(y_true, y_preds))

In [ ]:
# ----
# ### Evaluate the test set

# Infer results over validation set
num_test_examples = len(test_dataset)
y_raw_preds = np.empty((num_test_examples, num_classes))
y_preds = np.zeros((num_test_examples,))
y_true = np.zeros((num_test_examples,))

for i in tqdm(range(0, num_test_examples)):
    # Retrieve data
    idx = i  # Use index if evaluating over full dataset
    data, label = test_dataset[idx]
    # Infer
    data = torch.from_numpy(np.expand_dims(data, 0)).float().to(device)
    pred_tmp = test_model.predict(data)
    pred_tmp = pred_tmp.cpu().numpy() if torch.cuda.is_available() else pred_tmp
    # Argmax
    y_preds[i] = np.argmax(pred_tmp)
    # Store label
    y_true[i] = label

acc = np.sum(np.asarray(y_preds) == np.asarray(y_true)) / len(y_true)
plot_confusion_matrix(
    y_true,
    y_preds,
    classes = label_list,
    normalize = True,
    title="Test Set Modulations Confusion Matrix\nTotal Accuracy: {:.2f}%".format(
        acc * 100
    ),
    text=False,
    rotate_x_text=90,
    figsize=(16, 9),
)
plt.savefig(f"{checkpoint_dir}/test_result.png")

print("Test set classification report:")
print(classification_report(y_true, y_preds))